## Imports

In [ ]:
import json
import pathlib
import warnings
warnings.filterwarnings("ignore", message="Lazy modules") # Ignore lazy module warnings

import torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme() # Apply seaborn theme to plots

from hyperopt import hp, tpe, rand

from src.cross_validation import kfold_train
from src.data import load_dataset, get_dataloader
from src.evaluation import find_best_run, evaluate_checkpoint, find_best_hyperparameters
from src.hyperopt import optimize_hyperparameters
from src.logging import Logger
from src.model import WFDefectDetector
from src.training import train

## Configuration

In [ ]:
train_dataset = load_dataset(filepath="data/train", resolution=(64, 64))
test_dataset = load_dataset(filepath="data/test", resolution=(64, 64))
test_dataloader = get_dataloader(test_dataset, batch_size=8)

space = {
    "kernel_size_conv": hp.choice("kernel_size_conv", [3, 5, 7]),
    "kernel_size_pool": hp.choice("kernel_size_pool", [2, 3]),
    "num_base_channels": hp.choice("num_base_channels", [4, 8, 16, 32]),
    "num_conv_blocks": hp.choice("num_conv_blocks", [2, 3]),
    "activation": hp.choice("activation", ["tanh", "relu", "leakyrelu"]),
    "optimizer": hp.choice("optimizer", ["sgd", "rmsprop", "adam"]),
    "learning_rate": hp.uniform("learning_rate", 0.0001, 0.01),
}

device = "cuda" if torch.cuda.is_available() else "cpu"

## Exercise 1

In [ ]:
log_folder = pathlib.Path("runs_manual")
trial_id = "trial_before_hyperopt"

average_valid_loss = kfold_train(
    dataset=train_dataset,
    n_splits=5,
    batch_size=8,
    random_state=42,
    num_conv_blocks=2,
    num_base_channels=4,
    kernel_size_conv=3,
    kernel_size_pool=3,
    activation="relu",
    optimizer="adam",
    learning_rate=0.001,
    max_epochs=30,
    patience=3,
    device=device,
    log_folder=log_folder,
    trial_id=trial_id,
)

### Plot learning curves

In [ ]:
fig, (ax_left, ax_right) = plt.subplots(1, 2, dpi=250, figsize=(14, 6))
ax_left.set_xlabel("Epoch")
ax_right.set_xlabel("Epoch")
ax_left.set_title("Training/validation loss")
ax_right.set_title("Training/validation accuracy")

best_run_dir = find_best_run(log_folder / trial_id)

with open(best_run_dir / "logs.json", "r") as file:
    logs = json.load(file)

ax_left.plot(logs["train_loss"], label="Training loss")
ax_left.plot(logs["valid_loss"], label="Validation loss")
ax_right.plot(logs["train_accuracy"], label="Training accuracy")
ax_right.plot(logs["valid_accuracy"], label="Validation accuracy")

ax_left.legend()
ax_right.legend()

fig.savefig("plots/exercise-1-unoptimized.png", bbox_inches="tight")

### Run model on testing set

In [ ]:
evaluate_checkpoint(run_dir=best_run_dir, test_dataloader=test_dataloader)

## Exercise 2

In [ ]:
log_folder = pathlib.Path("runs_hyperopt_random")

optimize_hyperparameters(
    dataset=train_dataset,
    space=space,
    algo=rand.suggest,
    max_evals=50,
    n_splits=5,
    batch_size=8,
    random_state=42,
    max_epochs=30,
    patience=3,
    device=device,
    log_folder=log_folder,
)

print("Best hyperparameters:")
find_best_hyperparameters(log_folder)

In [ ]:
log_folder = pathlib.Path("runs_hyperopt_tpe")

optimize_hyperparameters(
    dataset=train_dataset,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    n_splits=5,
    batch_size=8,
    random_state=42,
    max_epochs=30,
    patience=3,
    device=device,
    log_folder=log_folder,
)

print("Best hyperparameters:")
find_best_hyperparameters(log_folder)

## Exercise 3

In [ ]:
log_folder = pathlib.Path("runs_manual")
trial_id = "trial_after_hyperopt"

average_valid_loss = kfold_train(
    dataset=train_dataset,
    n_splits=5,
    batch_size=8,
    random_state=42,
    num_conv_blocks=2,
    num_base_channels=4,
    kernel_size_conv=3,
    kernel_size_pool=3,
    activation="relu",
    optimizer="adam",
    learning_rate=0.001,
    max_epochs=30,
    patience=3,
    device=device,
    log_folder=log_folder,
    trial_id=trial_id,
)

### Plot learning curves

In [ ]:
fig, (ax_left, ax_right) = plt.subplots(1, 2, dpi=250, figsize=(14, 6))
ax_left.set_xlabel("Epoch")
ax_right.set_xlabel("Epoch")
ax_left.set_title("Training/validation loss")
ax_right.set_title("Training/validation accuracy")

best_run_dir = find_best_run(log_folder / trial_id)

with open(best_run_dir / "logs.json", "r") as file:
    logs = json.load(file)

ax_left.plot(logs["train_loss"], label="Training loss")
ax_left.plot(logs["valid_loss"], label="Validation loss")
ax_right.plot(logs["train_accuracy"], label="Training accuracy")
ax_right.plot(logs["valid_accuracy"], label="Validation accuracy")

ax_left.legend()
ax_right.legend()

fig.savefig("plots/exercise-3-optimized.png", bbox_inches="tight")

### Run model on testing set

In [ ]:
evaluate_checkpoint(run_dir=best_run_dir, test_dataloader=test_dataloader)